In [1]:
import pandas as pd
df = pd.read_csv('stroke.csv')
df= df[df['Comorbidities']==False]

df['TPA Time'] = pd.to_datetime(df['TPA Time'])
df['Emergency Dept Time'] = pd.to_datetime(df['Emergency Dept Time'])
df['TPA_Done'] = df['TPA Time'].notna().astype(int)
df['TimeForTPA'] = (df['TPA Time'] - df['Emergency Dept Time']).dt.total_seconds() / 60

In [2]:
df['ICU Checkout Time'] = pd.to_datetime(df['ICU Checkout Time'])
df['ICU Arrival Time'] = pd.to_datetime(df['ICU Arrival Time'])
df['ICUStay(Days)'] = (df['ICU Checkout Time'] - df['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)
df['ICUStay_Flag'] = df['ICU Arrival Time'].notna().astype(int)
df['Admission Time'] = pd.to_datetime(df['ICU Checkout Time'])
df['ICU Arrival Time'] = pd.to_datetime(df['ICU Arrival Time'])
df['ICUStay(Days)'] = (df['ICU Checkout Time'] - df['ICU Arrival Time']).dt.total_seconds() / (60 * 60 * 24)


In [3]:
def categorize_age(age):
    if age <= 45:
        return '0-45'
    elif 45 < age <= 60:
        return '45-60'
    elif 60 < age <=70:
        return '60-70'
    elif 70 < age <= 80:
        return '70-80'
    else:
        return '80+'

# Apply the function to create a new column 'Age Group'
df['Age Group'] = df['Age'].apply(categorize_age)
df['Age Group'].value_counts()


Age Group
45-60    668
60-70    662
70-80    351
0-45     252
80+      178
Name: count, dtype: int64

In [4]:
import matplotlib.pyplot as plt
import squarify
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact


age_groups = sorted(df['Age Group'].unique())
color_mapping = {
    (0, 0): '#1f77b4',  # Non-ICU & No TPA # blue
    (0, 1): '#ff7f0e',  # Non-ICU & TPA - orange
    (1, 0): '#2ca02c',  # ICU & No TPA green
    (1, 1): '#d62728',  # ICU & TPA red
}

@interact
def plot_treemap(selected_age_groups=widgets.SelectMultiple(options=age_groups, value=age_groups, description="Age Groups")):
    # Filter DataFrame based on selected age groups
    filtered_df = df[df['Age Group'].isin(selected_age_groups)]
    
    # Prepare the data
    icu_tpa = filtered_df.groupby(['ICUStay_Flag', 'TPA_Done']).size().reset_index(name='counts')
    icu_tpa['label'] = icu_tpa.apply(lambda x: f"{'ICU' if x['ICUStay_Flag'] == 1 else 'Non-ICU'} - {'TPA' if x['TPA_Done'] == 1 else 'No TPA'}\n{x['counts']}", axis=1)
    sizes = icu_tpa['counts'].values
    labels = icu_tpa['label'].values

    # Assign colors based on the defined mapping
    colors = [color_mapping[(row['ICUStay_Flag'], row['TPA_Done'])] for index, row in icu_tpa.iterrows()]

    # Plot
    plt.figure(figsize=(10, 8))
    squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=colors)
    plt.axis('off')
    plt.show()


interactive(children=(SelectMultiple(description='Age Groups', index=(0, 1, 2, 3, 4), options=('0-45', '45-60'…